In [2]:
import torch

# Force CPU usage
device = torch.device("cpu")
print(f"Using device: {device}")


Using device: cpu


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

df = pd.read_csv(r"C:\Users\srevatshen\imdb-emotion-classifier\data\imdb_reviews.csv")
df = df[['review', 'sentiment']].dropna()

label_encoder = LabelEncoder()
df['sentiment'] = label_encoder.fit_transform(df['sentiment'])

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['review'].tolist(), df['sentiment'].tolist(), test_size=0.2, random_state=42)


In [5]:
from torch.utils.data import Dataset

from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        encodings = self.tokenizer(
            self.texts[idx],
            truncation=True,
            padding='max_length',
            max_length=self.max_len,
            return_tensors="pt"
        )
        return {
            'input_ids': encodings['input_ids'].squeeze(),
            'attention_mask': encodings['attention_mask'].squeeze(),
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }


In [6]:
from torch.utils.data import DataLoader

train_dataset = SentimentDataset(train_texts, train_labels, tokenizer)
val_dataset = SentimentDataset(val_texts, val_labels, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)


In [7]:
from transformers import BertForSequenceClassification
from torch.optim import AdamW
from tqdm import tqdm
import time

# Start time
start_time = time.time()

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
model.to(device)

optimizer = AdamW(model.parameters(), lr=2e-5)

epochs = 1
for epoch in range(epochs):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = torch.argmax(logits, dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    acc = correct / total * 100
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1} - Loss: {avg_loss:.4f} - Accuracy: {acc:.2f}%")

# End time
end_time = time.time()
print(f"Training time on CPU: {(end_time - start_time):.2f} seconds")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 1/1: 100%|█████████████████████████████████████████████████████████████████████| 200/200 [56:10<00:00, 16.85s/it]

Epoch 1 - Loss: 0.4913 - Accuracy: 74.94%
Training time on CPU: 3371.04 seconds


In [8]:
model.save_pretrained("saved_model_cpu/")
tokenizer.save_pretrained("saved_model_cpu/")


('saved_model_cpu/tokenizer_config.json',
 'saved_model_cpu/special_tokens_map.json',
 'saved_model_cpu/vocab.txt',
 'saved_model_cpu/added_tokens.json')

In [6]:
from sklearn.metrics import classification_report, accuracy_score
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from tqdm import tqdm

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the saved model and tokenizer
model = BertForSequenceClassification.from_pretrained("saved_model_cpu/")
tokenizer = BertTokenizer.from_pretrained("saved_model_cpu/")
model.to(device)

# Evaluation function
def evaluate(model, dataloader, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Accuracy
    acc = accuracy_score(all_labels, all_preds)
    print(f"\nValidation Accuracy: {acc * 100:.2f}%")

    # Classification report
    print("\nClassification Report:")
    print(classification_report(all_labels, all_preds, target_names=['Negative', 'Positive']))

# Call the evaluation
# val_loader should already be defined and preprocessed using tokenizer
evaluate(model, val_loader, device)


Evaluating: 100%|██████████████████████████████████████████████████████████████████████| 50/50 [01:01<00:00,  1.23s/it]


Validation Accuracy: 87.75%

Classification Report:
              precision    recall  f1-score   support

    Negative       0.90      0.86      0.88       207
    Positive       0.85      0.90      0.88       193

    accuracy                           0.88       400
   macro avg       0.88      0.88      0.88       400
weighted avg       0.88      0.88      0.88       400

